# Advent of Code 2022 - Day 7 of 25

## Problem 1:

--- Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
cd / switches the current directory to the outermost directory, /.
ls means list. It prints out all of the files and directories immediately contained by the current directory:
123 abc means that the current directory contains a file named abc with size 123.
dir xyz means that the current directory contains a directory named xyz.
Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
Directory d has total size 24933642.
As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.
To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?



In [74]:
import re

with open('data/aoc_2022_day7.txt') as f:
    lines = [line.replace('\n', '') for line in f]
    
lines[:20]


['$ cd /',
 '$ ls',
 'dir fpljqj',
 '171526 ghtzhjwf.nls',
 'dir gsdsbld',
 'dir hbmjtb',
 '296801 mjfjqw.ccv',
 'dir nfn',
 'dir qmrsvfvw',
 '102565 qqjnqb.chd',
 'dir svgbqd',
 '$ cd fpljqj',
 '$ ls',
 '153563 ghtzhjwf.nls',
 '243252 gsvjgj.jsm',
 '154134 hghnrbqg.rzb',
 '$ cd ..',
 '$ cd gsdsbld',
 '$ ls',
 'dir npmncvhh']

In [ ]:
class directory:
    
    def __init__(self, directory_name, parent_directory, level):
        self.directory_name = directory_name
        self.parent_directory = parent_directory
        self.directory_level = level
        self.contents = []
        
        
    def get_content_size():
        pass
    

In [ ]:
pattern_cd = r'^[$] (cd) (.*)'
pattern_ls = r'^[$] (ls)'
pattern_file = r'^(\d*) (.*)'
pattern_dir = r'^(dir) (.*)'

In [79]:

def parse_commands(lines, verbose=False):
    
    folder_structure = {}
    folder_structure['home'] = []
    current_folder = 'Start'


    for line in lines:

        if verbose:
            print(f'The line is: {line}')
            print(f'Current folder_structure is {folder_structure}')
            print(f'Current Working Directory is {current_folder}\n')

        change_dir = re.findall(pattern_cd, line)
        list_files = re.findall(pattern_ls, line)
        file = re.findall(pattern_file, line)
        directory = re.findall(pattern_dir, line)

        if change_dir:

            if change_dir[0][1] == '/':
                
                if verbose:
                    print(f'Changing directory from {current_folder} to HOME')
                current_folder = change_dir[0][1]

            elif change_dir[0][1] == '..':
                if verbose:
                    print('Going up a level')
                for key, values in folder_structure.items():
                    if current_folder in values:
                        if verbose:
                            print(f'Changing folder from {current_folder} to {key}')
                        current_folder = key

            else:
                
                if verbose:
                    print(f'Changing directory from {current_folder} to {change_dir[0][1]}')
                current_folder = change_dir[0][1]

        elif list_files:

            if verbose:
                print(f'Listing files and directories in {current_folder}')


        elif file:

            if verbose:
                print(f'Found file {file[0][1]} in {current_folder}')

            if current_folder == '/':
                folder_structure['home'].append((int(file[0][0]), file[0][1]))
            else:
                folder_structure[current_folder].append((int(file[0][0]), file[0][1]))

        elif directory:

            if verbose:
                print(f'Found a directory {directory[0][1]}')

            if directory[0][1] in folder_structure:

                if current_folder == '/':
                    folder_structure['home'].append(directory[0][1])
                else:
                    folder_structure[current_folder].append(directory[0][1])

            else:
                
                if verbose:
                    print('Directory not previously noted, adding to folder_structure.')
                folder_structure[directory[0][1]] = []

                # Add knowledge of directory 
                if current_folder != '/':
                    folder_structure[current_folder].append(directory[0][1])
                else:
                    folder_structure['home'].append(directory[0][1])


        else:
            if verbose:
                print('Encountered unknown command')
                
    return folder_structure


In [81]:
folder_structure = parse_commands(lines, verbose=False)


In [82]:
folder_structure['home']

['fpljqj',
 (171526, 'ghtzhjwf.nls'),
 'gsdsbld',
 'hbmjtb',
 (296801, 'mjfjqw.ccv'),
 'nfn',
 'qmrsvfvw',
 (102565, 'qqjnqb.chd'),
 'svgbqd']

In [84]:
folders = []

for key, values in folder_structure.items():
    
    #print(f'Key is {key}')
    folders.append(key)
    

In [97]:
folders

['home',
 'fpljqj',
 'gsdsbld',
 'hbmjtb',
 'nfn',
 'qmrsvfvw',
 'svgbqd',
 'npmncvhh',
 'sqtnlr',
 'vzndpc',
 'mphhrqf',
 'rnmvggfd',
 'qlcfs',
 'zwsnwvnv',
 'htwl',
 'lqjhfdch',
 'nqfdhlcg',
 'zfl',
 'zlqgr',
 'dhjfqv',
 'crs',
 'whrm',
 'pjzpqs',
 'smz',
 'tdwqpg',
 'hwlhshtr',
 'czzrdvc',
 'fcsjqg',
 'fwdprwpq',
 'tvglfb',
 'blplbjpw',
 'sphpbjt',
 'zcgfg',
 'gqvmv',
 'fpglbth',
 'pfjz',
 'ngr',
 'fdhd',
 'hwsqqt',
 'lndmtm',
 'sgt',
 'tgjszvsg',
 'ztrvv',
 'hscwh',
 'wjv',
 'qgz',
 'fzbppql',
 'jpqqr',
 'lzq',
 'pjjbmllm',
 'bzcfzh',
 'vsrgqmlt',
 'zbthlb',
 'hjp',
 'twfw',
 'csmmbjhp',
 'tvfhvbp',
 'fghl',
 'lgpwpmzp',
 'sqjqg',
 'ctr',
 'tgfvcpp',
 'zwtz',
 'bhq',
 'tcnv',
 'fqw',
 'mpz',
 'gntjg',
 'jfbhz',
 'vpgpz',
 'hdq',
 'hvcdpzr',
 'lth',
 'scncl',
 'jwhnt',
 'pmcpnqrr',
 'wvvzbt',
 'nwjzld',
 'tfdcg',
 'vwlbtgnh',
 'lcdfdmlj',
 'pthwnf',
 'lnwtl',
 'dmbf',
 'ndbcrvw',
 'shsgqzn',
 'zpggmccr',
 'bql',
 'dcldv',
 'fsrznscl',
 'prcgb',
 'bcdrv',
 'jcffb',
 'jzl',
 'nzlv',
 

In [104]:
folder_filesizes={}
for folder in folders:
    folder_filesizes[folder] = 0
    for item in folder_structure[folder]:
        #print(f'Item {item} is of type {type(item)}')
        if type(item) == tuple:
            folder_filesizes[folder] += item[0]
            
#folder_filesizes     

{'home': 570892,
 'fpljqj': 3704857,
 'gsdsbld': 0,
 'hbmjtb': 493357,
 'nfn': 0,
 'qmrsvfvw': 3536113,
 'svgbqd': 69927,
 'npmncvhh': 502397,
 'sqtnlr': 959283,
 'vzndpc': 183360,
 'mphhrqf': 111207,
 'rnmvggfd': 278122,
 'qlcfs': 680431,
 'zwsnwvnv': 363633,
 'htwl': 268134,
 'lqjhfdch': 21479,
 'nqfdhlcg': 2252743,
 'zfl': 1386843,
 'zlqgr': 0,
 'dhjfqv': 135074,
 'crs': 220281,
 'whrm': 279796,
 'pjzpqs': 369179,
 'smz': 249693,
 'tdwqpg': 157907,
 'hwlhshtr': 90468,
 'czzrdvc': 0,
 'fcsjqg': 81548,
 'fwdprwpq': 477376,
 'tvglfb': 383166,
 'blplbjpw': 2835,
 'sphpbjt': 31414,
 'zcgfg': 424863,
 'gqvmv': 175614,
 'fpglbth': 58850,
 'pfjz': 197076,
 'ngr': 0,
 'fdhd': 168253,
 'hwsqqt': 174820,
 'lndmtm': 0,
 'sgt': 525763,
 'tgjszvsg': 98354,
 'ztrvv': 675282,
 'hscwh': 0,
 'wjv': 1278908,
 'qgz': 281901,
 'fzbppql': 29399,
 'jpqqr': 45275,
 'lzq': 180833,
 'pjjbmllm': 247746,
 'bzcfzh': 270737,
 'vsrgqmlt': 0,
 'zbthlb': 0,
 'hjp': 20745,
 'twfw': 0,
 'csmmbjhp': 275130,
 'tvfhvbp'

## Problem 2


In [92]:
type((102565, 'qqjnqb.chd'))

tuple